<a href="https://colab.research.google.com/github/NathanielRose/eeg-294/blob/master/Test_Notebook_Vilde_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Test Notebook Vilde: RCNN**



Trying to reproduce RCNN model example.

# **Setup**

In [1]:
!pip install git+git://github.com/gumpy-bci/gumpy.git

  Cloning git://github.com/gumpy-bci/gumpy.git to /tmp/pip-req-build-956tiuad
  Running command git clone -q git://github.com/gumpy-bci/gumpy.git /tmp/pip-req-build-956tiuad
  Created wheel for gumpy: filename=gumpy-0.5.0-cp36-none-any.whl size=32999 sha256=a46788dd8b949732f3f75adc9e8e1a2f30f722a7c6313759e72feb144a47ec00
  Stored in directory: /tmp/pip-ephem-wheel-cache-v3w540ij/wheels/cf/ee/4d/dfa04a0d04ffb1ca36f436f3fef4817e9e23bfec03dd772913
Successfully built gumpy


In [3]:
!git clone https://github.com/NathanielRose/eeg-294.git

fatal: destination path 'eeg-294' already exists and is not an empty directory.


#### **Importing packages**


Importing module:

In [4]:
%reset
%matplotlib inline

from __future__ import print_function
import os; os.environ["THEANO_FLAGS"] = "device=gpu0"
import os.path
from datetime import datetime
import sys

import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import pandas as pd

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


Importing gumpy modules:

In [5]:
sys.path.append('gumpy/gumpy/')
import gumpy

sys.path.append('eeg-294/gumpy-deeplearning')
import models

sys.path.append('eeg-294/gumpy-deeplearning/models/')
import utils

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


#### **Setup parameters for the model and data**

In [0]:
DEBUG = True
######
## the RCNN FLAG because It does not use spectrograms but rather it uses directly the raw signals.
# so the data have something different compared to the spectrogram so you have to activte the flag.
######
RCNN_FLAG = True
###########


CLASS_COUNT = 2
DROPOUT = 0.2   # dropout rate in float

# parameters for filtering data
FS = 250
LOWCUT = 2
HIGHCUT = 60
ANTI_DRIFT = 0.5
CUTOFF = 50.0 # freq to be removed from signal (Hz) for notch filter
Q = 30.0  # quality factor for notch filter
W0 = CUTOFF/(FS/2)
AXIS = 0

#set random seed
SEED = 42
KFOLD = 5

#### **Load raw data**

In [8]:
%%bash
%%capture 

export fileid=1DS64UOrt9wzswke5wiFtat94-s1YcRmb
export filename=grazdata.zip


wget --save-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='$fileid -O- \
     | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt

wget --load-cookies cookies.txt -O $filename \
     'https://docs.google.com/uc?export=download&id='$fileid'&confirm='$(<confirm.txt)

bash: line 1: fg: no job control
--2019-11-29 03:01:38--  https://docs.google.com/uc?export=download&id=1DS64UOrt9wzswke5wiFtat94-s1YcRmb
Resolving docs.google.com (docs.google.com)... 74.125.195.102, 74.125.195.101, 74.125.195.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘STDOUT’

     0K ...                                                    7.90M=0s

2019-11-29 03:01:38 (7.90 MB/s) - written to stdout [3248]

--2019-11-29 03:01:38--  https://docs.google.com/uc?export=download&id=1DS64UOrt9wzswke5wiFtat94-s1YcRmb&confirm=7BTr
Resolving docs.google.com (docs.google.com)... 74.125.195.101, 74.125.195.113, 74.125.195.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-94-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7def

In [9]:
!unzip grazdata.zip

Archive:  grazdata.zip
mapname:  conversion of  failed
replace B03T.mat? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
 extracting: B03T.mat                
 extracting: B05E.mat                
 extracting: B04T.mat                
 extracting: B01E.mat                
 extracting: B04E.mat                
 extracting: B06T.mat                
 extracting: B08T.mat                
 extracting: B07T.mat                
 extracting: B02T.mat                
 extracting: B03E.mat                
 extracting: B09T.mat                
 extracting: B05T.mat                
 extracting: B09E.mat                
 extracting: B01T.mat                
 extracting: B06E.mat                
 extracting: B07E.mat                
 extracting: B08E.mat                
 extracting: B02E.mat                


In [0]:
#load data
subject = 'B01'

grazb_data = gumpy.data.GrazB('', subject) # Initializing the data-structure
# grazb_data.load()


#### **Data preprocessing**
Section 3.2, first paragraph.

In [11]:
# now that the dataset is setup, we can load the data. This will be handled from within the utils function,
# which will first load the data and subsequently filter it using a notch and a bandpass filter.
# the utility function will then return the training data.
x_train, y_train = utils.load_preprocess_data(grazb_data, True, LOWCUT, HIGHCUT, W0, Q, ANTI_DRIFT, CLASS_COUNT, CUTOFF, AXIS, FS)

Band-pass filtering the data in frequency range from 2.0 Hz to 60.0 Hz... 
Data loaded and processed successfully!


#### **Augmented data**
Section 3.2, second paragraph.


In [0]:
x_augmented, y_augmented = gumpy.signal.sliding_window(data = x_train[:,:,:],
                                                          labels = y_train[:,:],
                                                          window_sz = 4 * FS,
                                                          n_hop = FS // 10,
                                                          n_start = FS * 1)
x_subject = x_augmented
y_subject = y_augmented
x_subject = np.rollaxis(x_subject, 2, 1)

###**Importing modules manually**
To fix a few errors in the code.


#### **.model**
 

In [0]:
import os
from abc import ABC, abstractmethod
import keras
from keras.models import model_from_json
from keras.callbacks import CSVLogger, ModelCheckpoint


class Model(ABC):
    """An abstract deep learning model.
    The abstract class functions as a facade for the backend. Although
    gumpy-deeplearning currently uses keras, it is possible that future releases
    may use different front- or backends. The Model ABC should represent the
    baseline for any such model.
    For more information about the reason behind ``Model``, see https://xkcd.com/927/
    """

    def __init__(self, name):
        self.name = name
        self.model = None

    @abstractmethod
    def create_model(self):
        pass

    @abstractmethod
    def fit(self):
        pass

    @abstractmethod
    def evaluate(self):
        pass

    @abstractmethod
    def from_json(self):
        pass




class KerasModel(Model):
    """ABC for Models that rely on keras.
    The ABC provides an implementation to generate callbacks to monitor the
    model and write the data to HDF5 files. The function ``fit`` simply forwards
    to the keras' ``fit``, but will enable monitoring if wanted.
    """

    def __init__(self, name):
        super(KerasModel, self).__init__(name)
        self.callbacks = None


    def get_callbacks(self):
        """Returns callbacks to monitor the model.
        """

        # save weights in an HDF5 file
        model_file = self.name + '_monitoring' + '.h5'
        checkpoint = ModelCheckpoint(model_file, monitor = 'val_loss',
                                     verbose = 0, save_best_only = True, mode = 'min')
        log_file = self.name + '.csv'
        csv_logger = CSVLogger(log_file, append = True, separator = ';')
        callbacks_list = [csv_logger] # callback list

        self.callbacks = callbacks_list
        return callbacks_list


    def fit(self, x, y, monitor=True, **kwargs):
        # TODO: allow user to specify filename
        if monitor and (self.callbacks is None):
            self.get_callbacks()

        if self.callbacks is not None:
            self.model.fit(x, y, **kwargs, callbacks=self.callbacks)
        else:
            self.model.fit(x, y, **kwargs)


    def evaluate(self, x, y, **kwargs):
        return self.model.evaluate(x, y, **kwargs)


    def from_json(self, model_file_name=None):
        try:
            # set the model_file_name if it is not passed to the function
            if model_file_name is None:
                model_file_name = self.name

            # load trained model
            model_path = model_file_name + ".json"
            if not os.path.isfile(model_path):
                raise IOError('file "%s" does not exist' %(model_path))
            model = model_from_json(open(model_path).read())

            # load weights of trained model
            model_weight_path = model_file + ".hdf5"
            if not os.path.isfile(model_weight_path):
                raise OSError('file "%s" does not exist' %(model_path))
            model.load_weights(model_weight_path)

            return model
        except IOError:
            print(IOError)
            return None

#### **RCNN**

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import BatchNormalization, Dropout, Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import merge, Conv2D, MaxPooling2D, Input
from keras.layers.advanced_activations import PReLU
from keras.layers import Activation
from keras.models import Model


class RCNN(KerasModel):

    # TODO: why is this called RCL?
    # Recurrent Convolutional Layer? 
    def RCL(self,l, a):
        # TODO: documentation

        # first convolutional layer
        conv1 = Conv2D(filters=128, kernel_size=(1, 9), strides=(1, 1), padding='same', data_format='channels_last',
                       init='he_normal')(l)
        bn1 = BatchNormalization(epsilon=0.000001)(conv1)
        relu1 = PReLU()(bn1)
        pool1 = MaxPooling2D(pool_size=(1, 4), strides=(1, 4), padding='valid', data_format='channels_last')(relu1)
        drop1 = Dropout(0)(pool1)

        # start first RCL layer
        # the second time convolution and stored for recurrent
        conv2 = Conv2D(filters=128, kernel_size=(1, 1), padding='same', init='he_normal')(drop1)
        bn2 = BatchNormalization(axis=1, epsilon=0.000001)(conv2)
        relu2 = PReLU()(bn2)

        # first recurrent for the first convolution
        conv2a = Conv2D(filters=128, kernel_size=(1, 9), padding='same', init='he_normal')
        conv2aa = conv2a(relu2)
        merged2a = merge([conv2, conv2aa], mode='sum')

        # second recurrent for the first convolution
        bn2a = BatchNormalization(axis=1, epsilon=0.000001)(merged2a)
        relu2a = PReLU()(bn2a)
        conv2b = Conv2D(filters=128, kernel_size=(1, 9), padding='same', weights=conv2a.get_weights())(relu2a)
        merged2b = merge([conv2, conv2b], mode='sum')

        # third recurrent for the first convolution
        bn2b = BatchNormalization(axis=1, epsilon=0.000001)(merged2b)
        relu2b = PReLU()(bn2b)
        conv2c = Conv2D(filters=128, kernel_size=(1, 9), padding='same', weights=conv2a.get_weights())(relu2b)
        merged2c = merge([conv2, conv2c], mode='sum')

        bn2c = BatchNormalization(axis=1, epsilon=0.000001)(merged2c)
        relu2c = PReLU()(bn2c)
        pool2 = MaxPooling2D(pool_size=(1, 4), strides=(1, 4), padding='valid', data_format='channels_last')(relu2c)
        drop2 = Dropout(0.2)(pool2)

        conv3 = Conv2D(filters=128, kernel_size=(1, 1), padding='same')(drop2)
        bn3 = BatchNormalization(axis=1, epsilon=0.000001)(conv3)
        relu3 = PReLU()(bn3)
        conv3a = Conv2D(filters=128, kernel_size=(1, 9), padding='same', init='he_normal')
        conv3aa = conv3a(relu3)
        merged3a = merge([conv3, conv3aa], mode='sum')

        bn3a = BatchNormalization(axis=1, epsilon=0.000001)(merged3a)
        relu3a = PReLU()(bn3a)
        conv3b = Conv2D(filters=128, kernel_size=(1, 9), padding='same', weights=conv3a.get_weights())(relu3a)
        merged3b = merge([conv3, conv3b], mode='sum')

        bn3b = BatchNormalization(axis=1, epsilon=0.000001)(merged3b)
        relu3b = PReLU()(bn3b)
        conv3c = Conv2D(filters=128, kernel_size=(1, 9), padding='same', weights=conv3a.get_weights())(relu3b)
        merged3c = merge([conv3, conv3c], mode='sum')

        bn3c = BatchNormalization(axis=1, epsilon=0.000001)(merged3c)
        relu3c = PReLU()(bn3c)
        pool3 = MaxPooling2D(pool_size=(1, 4), strides=(1, 4), padding='valid', data_format='channels_last')(relu3c)
        drop3 = Dropout(0.2)(pool3)

        conv4 = Conv2D(filters=128, kernel_size=(1, 1), padding='same', init='he_normal')(drop3)
        bn4 = BatchNormalization(axis=1, epsilon=0.000001)(conv4)
        relu4 = PReLU()(bn4)
        conv4a = Conv2D(filters=128, kernel_size=(1, 9), padding='same')
        conv4aa = conv4a(relu4)
        merged4a = merge([conv4, conv4aa], mode='sum')

        bn4a = BatchNormalization(axis=1, epsilon=0.000001)(merged4a)
        relu4a = PReLU()(bn4a)
        conv4b = Conv2D(filters=128, kernel_size=(1, 9), padding='same', weights=conv4a.get_weights())(relu4a)
        merged4b = merge([conv4, conv4b], mode='sum')

        bn4b = BatchNormalization(axis=1, epsilon=0.000001)(merged4b)
        relu4b = PReLU()(bn4b)
        conv4c = Conv2D(filters=128, kernel_size=(1, 9), padding='same', weights=conv4a.get_weights())(relu4b)
        merged4c = merge([conv4, conv4c], mode='sum')

        bn4c = BatchNormalization(axis=1, epsilon=0.000001)(merged4c)
        relu4c = PReLU()(bn4c)
        pool4 = MaxPooling2D(pool_size=(1, 4), strides=(1, 4), padding='valid', data_format='channels_last')(relu4c)
        drop4 = Dropout(0.2)(pool4)

        conv5 = Conv2D(filters=128, kernel_size=(1, 1), padding='same')(drop4)
        bn5 = BatchNormalization(axis=1, epsilon=0.000001)(conv5)
        relu5 = PReLU()(bn5)
        conv5a = Conv2D(filters=128, kernel_size=(1, 9), padding='same')
        conv5aa = conv5a(relu5)
        merged5a = merge([conv5, conv5aa], mode='sum')

        bn5a = BatchNormalization(axis=1, epsilon=0.000001)(merged5a)
        relu5a = PReLU()(bn5a)
        conv5b = Conv2D(filters=128, kernel_size=(1, 9), padding='same', weights=conv5a.get_weights())(relu5a)
        merged5b = merge([conv5, conv5b], mode='sum')

        bn5b = BatchNormalization(axis=1, epsilon=0.000001)(merged5b)
        relu5b = PReLU()(bn5b)
        conv5c = Conv2D(filters=128, kernel_size=(1, 9), padding='same', weights=conv5a.get_weights())(relu5b)
        merged5c = merge([conv5, conv5c], mode='sum')

        bn5c = BatchNormalization(axis=1, epsilon=0.000001)(merged5c)
        relu5c = PReLU()(bn5c)
        # pool5 = MaxPooling2D(pool_size=(1, 4), strides=(1, 4), padding='valid', data_format='channels_last')(relu5c)
        drop5 = Dropout(0.2)(relu5c)

        conv_relu = Activation('sigmoid')(drop5)

        # TODO: what is going on with this variable name?
        l1111 = Flatten()(conv_relu)
        out = Dense(a, activation='softmax')(l1111)

        return out

    # TODO: documentation
    def create_model(self, input_shape, print_summary=False, class_count = 2):
        """Create a new RCNN model instance"""

        changed_shape = (1,input_shape[1],input_shape[0])
        input_1 = Input(changed_shape)
        output = self.RCL(input_1, class_count) # class_count was a but gave error, and is thus changed
        model = Model(inputs=input_1, outputs=output)
        model.compile(loss='categorical_crossentropy',
                      optimizer='RMSprop',
                      metrics=['accuracy'])
        self.model = model
        return model

#### **Run the model**

In [0]:
from sklearn.model_selection import StratifiedKFold
import datetime
from keras.callbacks import ModelCheckpoint,TensorBoard,EarlyStopping
#from models import RCNN

In [0]:
#train data follos format [x, y].  Same with val_data
def run_model(model, model_name, epochs, train_data, val_split, batch_size, monitor_index):
    # save best weights
    if os.path.isdir('logs'):
      log_dir="logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    else:
      os.mkdir('logs')
      log_dir="logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    
    if not os.path.isdir('checkpoints'):
      os.mkdir('checkpoints')
    checkpointer = ModelCheckpoint('checkpoints/' + '%s.h5'%model_name, monitor=monitor_index, verbose=1,save_best_only=True, mode='auto', period=1)
    
    # stop if the model does not improve for 15 epoches
    stoppointer = EarlyStopping(monitor=monitor_index, min_delta=0, patience=15, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

    # run model
    history = model.fit(train_data[0], train_data[1], epochs=epochs,
                   shuffle=True,
                   batch_size=batch_size, validation_split = val_split,
                   callbacks=[checkpointer,stoppointer], verbose=2)
    return history


In [0]:
# saves model to json file saves model weights to h5 file
def save_model(model, model_name, working_dir):

    model_json = model.to_json()
    if os.path.isdir('models'):
      # save model to json
      with open(working_dir + 'models/' + "%s.json"%model_name, "w") as json_file:
        json_file.write(model_json)
    else:
      os.mkdir('models')
      with open(working_dir + 'models/' + "%s.json"%model_name, "w") as json_file:
        json_file.write(model_json)

    #save model weights
    model.save_weights(working_dir +'models/' + '%s_weights.h5'%model_name)


In [0]:
# draw the training curves
def draw_graphs(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,max(plt.ylim())])
    plt.title('Training and Validation Loss')
    plt.show()

In [25]:


# defining the K-fold cross validation 
kfold = StratifiedKFold(n_splits=KFOLD, shuffle=True, random_state=SEED)
cvscores = []
ii = 1
for train, test in kfold.split(x_subject, y_subject[:, 0]):
    print('Run ' + str(ii) + '...')

    # create callbacks
    model_name_str = 'GRAZ_CNN_STFT_3layer_' + \
                     '_run_' + str(ii)

    # initialize and create the model
    model = RCNN(model_name_str)
    model.create_model(x_subject.shape[1:], print_summary=True, class_count = CLASS_COUNT)

    #callbacks_list = model.get_callbacks()

    # fit model. If you specify monitor=True, then the model will create callbacks
    # and write its state to a HDF5 file

    if (RCNN_FLAG == True):
        x_subject = np.rollaxis(x_subject, 2, 1)
        x_subject = x_subject[:, np.newaxis, :, :]
        #print(x_subject.shape)
    model.fit(x_subject[train], 
              y_subject[train], 
              monitor=True,
              epochs=100,
              batch_size=256,
              verbose=0,
              validation_split=0.1)
              #callbacks = callbacks_list)
    #
    # run_model returns fitted model 
    #h = run_model(model.model, model_name_str,100, train, 0.1, 256, monitor_index)
    #save_model(model.model, model_name_str, '')

    draw_graphs(h)
    history.append(h) 
    
    # evaluate the model
    print('Evaluating model on test set...')
    scores = model.evaluate(x_subject[test], y_subject[test], verbose=0)
    print("Result on test set: %s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
    cvscores.append(scores[1] * 100)
    ii += 1

# print some evaluation statistics and write results to file
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
cv_all_subjects = np.asarray(cvscores)
print('Saving CV values to file....')
np.savetxt('GRAZ_CV_' + 'CNN_STFT_3layer_' + str(DROPOUT) + 'do' + '.csv',
           cv_all_subjects, delimiter=',', fmt='%2.4f')
print('CV values successfully saved!\n')





try:
  # print some evaluation statistics and write results to file
  print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
  cv_all_subjects = np.asarray(cvscores)
  print('Saving CV values to file....')
  np.savetxt('GRAZ_CV_' + 'RCNN_' + str(DROPOUT) + 'do'+'.csv', 
              cv_all_subjects, delimiter = ',', fmt = '%2.4f')
  print('CV values successfully saved!\n')
except:
  print ('could not save')

Run 1...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=128, kernel_size=(1, 9), strides=(1, 1), padding="same", data_format="channels_last", kernel_initializer="he_normal")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=128, kernel_size=(1, 1), padding="same", kernel_initializer="he_normal")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=128, kernel_size=(1, 9), padding="same", kernel_initializer="he_normal")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.6/dist-packages/keras/legacy/layers.py:460: UserWarning: The `Me

ValueError: ignored

In [58]:
RCL(self,l, a)
l = input_l
a = class_count

        # first convolutional layer
        conv1 = Conv2D(filters=128, kernel_size=(1, 9), strides=(1, 1), padding='same', data_format='channels_last',
                       init='he_normal')(l)
        bn1 = BatchNormalization(epsilon=0.000001)(conv1)
        relu1 = PReLU()(bn1)
        pool1 = MaxPooling2D(pool_size=(1, 4), strides=(1, 4), padding='valid', data_format='channels_last')(relu1)
        drop1 = Dropout(0)(pool1)

        # start first RCL layer
        # the second time convolution and stored for recurrent
        conv2 = Conv2D(filters=128, kernel_size=(1, 1), padding='same', init='he_normal')(drop1)
        bn2 = BatchNormalization(axis=1, epsilon=0.000001)(conv2)
        relu2 = PReLU()(bn2)

        # first recurrent for the first convolution
        conv2a = Conv2D(filters=128, kernel_size=(1, 9), padding='same', init='he_normal')
        conv2aa = conv2a(relu2)
        merged2a = merge([conv2, conv2aa], mode='sum')


(10044, 3, 1000)

In [71]:
keras.__version__

'2.2.5'

In [72]:
pip install --upgrade keras==2.0.5

     |████████████████████████████████| 225kB 5.1MB/s 
  Created wheel for keras: filename=Keras-2.0.5-cp36-none-any.whl size=253391 sha256=92114f9c50d5bf190d2fd0c8c2df3c3bb146218709183f2f8f3f0f1e4e5ad906
  Stored in directory: /root/.cache/pip/wheels/ee/09/df/2f10a3b36c903a620183396e24452e7de4dba01861b1d87cea
Successfully built keras
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.5 which is incompatible.
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [66]:
input_shape = (3, 1000)

changed_shape = (1,input_shape[1],input_shape[0])
input_1 = Input(changed_shape)

l = input_1
a = 2
conv1 = Conv2D(filters=128, kernel_size=(1, 9), strides=(1, 1), padding='same', data_format='channels_last',
                       init='he_normal')(l)

bn1 = BatchNormalization(epsilon=0.000001)(conv1)
relu1 = PReLU()(bn1)
pool1 = MaxPooling2D(pool_size=(1, 4), strides=(1, 4), padding='valid', data_format='channels_last')(relu1)
drop1 = Dropout(0)(pool1)

# start first RCL layer
# the second time convolution and stored for recurrent
conv2 = Conv2D(filters=128, kernel_size=(1, 1), padding='same', init='he_normal')(drop1)
bn2 = BatchNormalization(axis=1, epsilon=0.000001)(conv2)
relu2 = PReLU()(bn2)

# first recurrent for the first convolution
conv2a = Conv2D(filters=128, kernel_size=(1, 9), padding='same', init='he_normal')
conv2aa = conv2a(relu2)

merged2a = merge([conv2, conv2aa], mode='sum')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=128, kernel_size=(1, 9), strides=(1, 1), padding="same", data_format="channels_last", kernel_initializer="he_normal")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=128, kernel_size=(1, 1), padding="same", kernel_initializer="he_normal")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=128, kernel_size=(1, 9), padding="same", kernel_initializer="he_normal")`


TypeError: ignored